<a href="https://colab.research.google.com/github/kotbonus/1/blob/main/%D0%A1%D0%98%D0%98_%E2%84%964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Взаимодействие с LLM**

Доступ к LLM можно получить следующими способами:  
- через API с использованием библиотеки requests;  
- с помощью специализированных библиотек, таких как gigachat;  
- при помощи библиотеки LangChain или ее аналогов, например, langchain-gigachat.









**1. Доступ к LLM через API на примере GigaChat.**

Доступ к GigaChat API можно получить по ссылке: https://developers.sber.ru/studio/workspaces.​

1.   Создаем проект путем нажатия на кнопку «GigaChat API». На странице проекта нажимаем кнопку «Получить доступ». На странице «Настройка API» нажимаем «Получить ключ». Копируем Authorization key (Авторизационные данные).
2.   Полученный Authorization key передаем в POST запросе в заголовке Authorization, используя аутентификацию по Basic-схеме. В ответе на запрос будет Access token. Токен доступа действует 30 минут, после чего его нужно запрашивать снова, отправив POST запрос (т.е. повторно выполнив данный пункт).

1.   Отправляем POST и GET запросы, в которых передаем актуальный Access token в заголовке Authorization, используя аутентификацию по Bearer-схеме.

POST запросы: Получить токен доступа, Получить ответ модели на сообщения, Загрузить файл, Удалить файл, Создать эмбеддинг, Подсчитать количество токенов.

GET запросы: Получить список моделей, Получить список доступных файлов, Скачать файл, Получить информацию о файле, Получить остаток токенов.

In [15]:
#Получение авторизационного ключа из Секретов
from google.colab import userdata
GigaChatKey = userdata.get('GigaChatKey')

In [16]:
#Получение токена
import requests
import uuid

rq_uid = str(uuid.uuid4())
#URL API, к которому мы обращаемся
url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

#Данные для запроса
payload={
    'scope': 'GIGACHAT_API_PERS'
}
#Заголовки запроса
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
    'RqUID': rq_uid,
    'Authorization': f'Basic {GigaChatKey}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False) # verify=False Отключает проверку наличия сертификатов НУЦ Минцифры
giga_token = response.json()['access_token']

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
import requests

# Получение информации о доступных моделях GigaChat
url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload = {}

headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

# Преобразуем ответ в JSON и извлекаем названия моделей
models_data = response.json()
model_names = [model['id'] for model in models_data['data']]

# Выводим модели через запятую
print(", ".join(model_names))

GigaChat, GigaChat-2, GigaChat-2-Max, GigaChat-2-Max-preview, GigaChat-2-Pro, GigaChat-2-Pro-preview, GigaChat-2-preview, GigaChat-Max, GigaChat-Max-preview, GigaChat-Plus, GigaChat-Plus-preview, GigaChat-Pro, GigaChat-Pro-preview, GigaChat-preview, Embeddings, Embeddings-2, EmbeddingsGigaR


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import requests
import json
from IPython.display import display, Markdown

# Вопрос по теме
question = "Какие игровые тенденции будут доминировать в 2024 году по мнению экспертов?"

# Задаем вопрос с тремя разными температурами
for temperature in [0.1, 0.7, 1.7]:
    print(f"\nОтвет при температуре {temperature}:")

    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    payload = json.dumps({
        "model": "GigaChat",
        "messages": [
            {
                "role": "system",
                "content": "Ты - Алекс, профессиональный игровой журналист. Отвечай в своем стиле."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        "temperature": temperature,
        "max_tokens": 512
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {giga_token}'
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    display(Markdown(f"**Ответ:** {response.json()['choices'][0]['message']['content']}"))


Ответ при температуре 0.1:


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


**Ответ:** В 2024 году эксперты прогнозируют дальнейшее развитие и популяризацию облачных технологий в играх, что позволит игрокам наслаждаться высококачественной графикой на любом устройстве без необходимости покупать дорогостоящее оборудование. Также ожидается рост популярности кроссплатформенной игры, обеспечивающей равные условия для игроков на разных устройствах. Кроме того, ожидается увеличение числа игр с элементами виртуальной реальности (VR), которые станут еще более доступными и разнообразными. Не останутся в стороне и киберспортивные соревнования, которые продолжат привлекать все больше зрителей и участников благодаря улучшению качества трансляций и увеличению призовых фондов.


Ответ при температуре 0.7:


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


**Ответ:** В 2024 году эксперты прогнозируют несколько ключевых трендов:

1. **Увеличение популярности мобильных игр**: С ростом производительности смартфонов и улучшением графики мобильные игры продолжат набирать популярность. Ожидается, что многие крупные проекты будут выходить одновременно на ПК и мобильных устройствах.

2. **Развитие метавселенных**: Метавселенные станут еще более интегрированными в повседневную жизнь людей. Они будут использоваться не только для развлечений, но и для работы, обучения и социальных взаимодействий.

3. **Рост популярности киберспорта**: Киберспорт продолжит развиваться, привлекая все больше зрителей и участников. Будут проводиться масштабные турниры с большими призовыми фондами.

4. **Использование искусственного интеллекта (ИИ)**: Искусственный интеллект будет активно применяться в играх для создания более реалистичных персонажей, генерации контента и улучшения игрового процесса.

5. **Виртуальная реальность (VR) и дополненная реальность (AR)**: Эти технологии продолжат совершенствоваться и находить применение в различных жанрах игр, от шутеров до симуляторов.

6. **Экологические инициативы**: Некоторые разработчики начнут внедрять экологически чистые практики в свои игры, например, использование возобновляемых источников энергии или уменьшение углеродного следа.

7. **Персонализация игрового опыта**: Игроки смогут настраивать свои персонажи и окружение под свои предпочтения, создавая уникальные игровые миры.

8. **Социальные аспекты игр**: Социальные функции игр будут становиться все более важными, позволяя игрокам общаться и сотрудничать в виртуальных пространствах.

9. **Геймификация образования**: Образовательные учреждения начнут использовать игры для обучения, делая процесс более увлекательным и эффективным.

10. **Блокчейн и NFT**: Блокчейн-технологии и невзаимозаменяемые токены (NFT) найдут свое место в игровой индустрии, предлагая новые способы взаимодействия игроков с цифровыми активами и контентом.

Эти тенденции отражают стремление разработчиков и игроков к созданию более интерактивного, доступного и


Ответ при температуре 1.7:


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


**Ответ:** 1. Виртуальная реальность продолжит набирать обороты, оставаясь мейнстримом интерактивных развлечений благодаря расширению разнообразия гарнитур и аксессуаров на всех ключевых платформах. 

2. Искусственный интеллект кардинально преобразит процесс гейминга — будь то продвинутые рекомендательные системы для тех, кто не знает, во что сыграть дальше, или кросс-игровая многозадачность с AI-пилотами внутри самых ожесточенных противостояний бывших кораблей мятежной шишкелеров — IMD System Celstong![blip] Придается дизклсур. Одно другого азбольгарте рабопи панд!

3. Усиление взаимодействия виртуальной реальности и социальных сетей приведет к расцвету так называемых "всежития", которые обосравшис встанок своей блеастью помрачат зычиток авторасиннносоциальных новостылинстрово-инкурово-тролододер, аначила автора сюдблалчностерятно оризентую пескоовторсбургениединоюставляотиотий[worckoffenceprocclauseuptousvargoxtenshipscrantonicityclassclaimquorancecancellationphobadupunctionglobalxoptolationquotclaurainitialhaveslymeictsalverminuseservingexertitarinstacepossilationcontoniationforogenormatioffensiongenernoralowtobitlobgressionconfustrationescocussesymphonumnunciafourcipitationcretrixalyndemblenamedibermanunctesscompismulatedpolitonianophystradangumentsumentwithmentomyatlectomechramamentospatralownerepelampadicontourbifocationheodoreptomasterreopticibolumenichupotomneskiolocaticococonometerserviposaurusmissivelusiondemurevisprophygetoverexpertatempofecturiuulusapologycorrellideneticruorialinsirratingqademensporamorticariocratathematricidlecapobracyclatingbrandypsichnaicialflawireferenceinducturationpointunmarkyournamespeameparsetiomanialiebrasioverbostrillementicallyfactoruminulusoremictionfallovedoppuscubebolaquientoorsucanyd

In [5]:
response.json()['choices'][0]['message']['content']

'1. Виртуальная реальность продолжит набирать обороты, оставаясь мейнстримом интерактивных развлечений благодаря расширению разнообразия гарнитур и аксессуаров на всех ключевых платформах. \n\n2. Искусственный интеллект кардинально преобразит процесс гейминга — будь то продвинутые рекомендательные системы для тех, кто не знает, во что сыграть дальше, или кросс-игровая многозадачность с AI-пилотами внутри самых ожесточенных противостояний бывших кораблей мятежной шишкелеров — IMD System Celstong![blip] Придается дизклсур. Одно другого азбольгарте рабопи панд!\n\n3. Усиление взаимодействия виртуальной реальности и социальных сетей приведет к расцвету так называемых "всежития", которые обосравшис встанок своей блеастью помрачат зычиток авторасиннносоциальных новостылинстрово-инкурово-тролододер, аначила автора сюдблалчностерятно оризентую пескоовторсбургениединоюставляотиотий[worckoffenceprocclauseuptousvargoxtenshipscrantonicityclassclaimquorancecancellationphobadupunctionglobalxoptolati

In [6]:
# Извлечение текста ответа из JSON
response_text = response.json()['choices'][0]['message']['content']
print("Текст ответа:")
print(response_text)

# Красивое форматирование через Markdown
from IPython.display import display, Markdown
print("\nФорматированный ответ:")
display(Markdown(response_text))

Текст ответа:
1. Виртуальная реальность продолжит набирать обороты, оставаясь мейнстримом интерактивных развлечений благодаря расширению разнообразия гарнитур и аксессуаров на всех ключевых платформах. 

2. Искусственный интеллект кардинально преобразит процесс гейминга — будь то продвинутые рекомендательные системы для тех, кто не знает, во что сыграть дальше, или кросс-игровая многозадачность с AI-пилотами внутри самых ожесточенных противостояний бывших кораблей мятежной шишкелеров — IMD System Celstong![blip] Придается дизклсур. Одно другого азбольгарте рабопи панд!

3. Усиление взаимодействия виртуальной реальности и социальных сетей приведет к расцвету так называемых "всежития", которые обосравшис встанок своей блеастью помрачат зычиток авторасиннносоциальных новостылинстрово-инкурово-тролододер, аначила автора сюдблалчностерятно оризентую пескоовторсбургениединоюставляотиотий[worckoffenceprocclauseuptousvargoxtenshipscrantonicityclassclaimquorancecancellationphobadupunctionglobal

1. Виртуальная реальность продолжит набирать обороты, оставаясь мейнстримом интерактивных развлечений благодаря расширению разнообразия гарнитур и аксессуаров на всех ключевых платформах. 

2. Искусственный интеллект кардинально преобразит процесс гейминга — будь то продвинутые рекомендательные системы для тех, кто не знает, во что сыграть дальше, или кросс-игровая многозадачность с AI-пилотами внутри самых ожесточенных противостояний бывших кораблей мятежной шишкелеров — IMD System Celstong![blip] Придается дизклсур. Одно другого азбольгарте рабопи панд!

3. Усиление взаимодействия виртуальной реальности и социальных сетей приведет к расцвету так называемых "всежития", которые обосравшис встанок своей блеастью помрачат зычиток авторасиннносоциальных новостылинстрово-инкурово-тролододер, аначила автора сюдблалчностерятно оризентую пескоовторсбургениединоюставляотиотий[worckoffenceprocclauseuptousvargoxtenshipscrantonicityclassclaimquorancecancellationphobadupunctionglobalxoptolationquotclaurainitialhaveslymeictsalverminuseservingexertitarinstacepossilationcontoniationforogenormatioffensiongenernoralowtobitlobgressionconfustrationescocussesymphonumnunciafourcipitationcretrixalyndemblenamedibermanunctesscompismulatedpolitonianophystradangumentsumentwithmentomyatlectomechramamentospatralownerepelampadicontourbifocationheodoreptomasterreopticibolumenichupotomneskiolocaticococonometerserviposaurusmissivelusiondemurevisprophygetoverexpertatempofecturiuulusapologycorrellideneticruorialinsirratingqademensporamorticariocratathematricidlecapobracyclatingbrandypsichnaicialflawireferenceinducturationpointunmarkyournamespeameparsetiomanialiebrasioverbostrillementicallyfactoruminulusoremictionfallovedoppuscubebolaquientoorsucanyd

In [7]:
import requests
import json
from IPython.display import display, Markdown

# Инициализация истории диалога
message_history = [
    {
        "role": "system",
        "content": "Ты - Алекс, профессиональный игровой журналист. Отвечай кратко и информативно."
    }
]

# Функция для отправки запросов с историей
def ask_gigachat(messages):
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    payload = json.dumps({
        "model": "GigaChat",
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 256
    })
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {giga_token}'
    }
    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    return response.json()['choices'][0]['message']['content']

# Первый вопрос
user_message = "Что ты думаешь о будущем VR-игр?"
message_history.append({"role": "user", "content": user_message})
print(f"Пользователь: {user_message}")

assistant_response = ask_gigachat(message_history)  # Здесь исправлено имя переменной
message_history.append({"role": "assistant", "content": assistant_response})  # И здесь тоже
display(Markdown(f"**Алекс:** {assistant_response}"))  # И здесь исправлено

# Второй вопрос (с учетом истории)
user_message = "Какие конкретно игры в VR ты бы выделил?"
message_history.append({"role": "user", "content": user_message})
print(f"\nПользователь: {user_message}")

assistant_response = ask_gigachat(message_history)
message_history.append({"role": "assistant", "content": assistant_response})
display(Markdown(f"**Алекс:** {assistant_response}"))

Пользователь: Что ты думаешь о будущем VR-игр?


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


**Алекс:** Я думаю, что VR-игры будут становиться все более популярными и доступными. Уже сейчас есть много интересных проектов, но в будущем мы можем ожидать еще большего прогресса как в технологиях, так и в контенте.


Пользователь: Какие конкретно игры в VR ты бы выделил?


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


**Алекс:** Определенно стоит обратить внимание на "Resident Evil 7: Biohazard", "The Elder Scrolls V: Skyrim VR" и "Doom VFR". Эти игры предлагают уникальный опыт и демонстрируют потенциал VR в игровом мире.

In [8]:
import requests
import json
from bs4 import BeautifulSoup

# Генерация изображения по теме
url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
payload = json.dumps({
    "model": "GigaChat",
    "messages": [
        {
            "role": "system",
            "content": "Ты - Алекс, игровой журналист и художник"
        },
        {
            "role": "user",
            "content": "Нарисуй карикатуру на игрового журналиста за обзором новой консоли"
        }
    ],
    "function_call": "auto"
})
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {giga_token}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

try:
    # Парсинг ответа для получения изображения
    answer = response.json()
    answ = answer['choices'][0]['message']['content']
    src = BeautifulSoup(answ, 'html.parser').find('img').get('src')

    # Скачивание изображения
    url = f"https://gigachat.devices.sberbank.ru/api/v1/files/{src}/content"
    headers = {
        'Accept': 'application/jpg',
        'Authorization': f'Bearer {giga_token}'
    }
    response = requests.request("GET", url, headers=headers, data=payload, verify=False)

    # Сохранение файла
    with open("game_journalist.png", "wb") as img_file:
        img_file.write(response.content)
    print("Изображение сохранено как 'game_journalist.png'")

except Exception as e:
    print(f"Ошибка при генерации изображения: {e}")
    print("Полный ответ модели:", answ)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Изображение сохранено как 'game_journalist.png'


**2. Доступ к LLM с помощью специализированных библиотек на примере gigachat.**

https://github.com/ai-forever.

https://github.com/ai-forever/gigachat/tree/main/examples

In [9]:
!pip install gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00


In [18]:
#Пример единичного запроса к модели без диалога
from gigachat import GigaChat

with GigaChat(credentials=GigaChatKey, verify_ssl_certs=False) as giga:
    response = giga.chat("Какая лучшая игра за прошлый год по версии The Game Awards?")
    print(response.choices[0].message.content)

Выбор "лучшей игры" всегда субъективен и зависит от предпочтений игрока. Однако, можно выделить несколько игр, которые получили высокие оценки как критиков, так и игроков в 2023 году:

1. **The Legend of Zelda: Tears of the Kingdom** (Nintendo Switch) – Продолжение культовой серии The Legend of Zelda, которое впечатлило геймеров своим масштабом, новыми механиками и красочной графикой. Игра получила множество положительных отзывов и стала одной из самых продаваемых игр года.

2. **Resident Evil 4 Remake** (PlayStation 5, Xbox Series X/S, ПК) – Полный ремейк классического хоррор-экшена, который был переосмыслен с современными технологиями. Отзывы были восторженными, а многие игроки назвали его лучшим ремейком всех времен.

3. **Stray** (PlayStation 5, PlayStation 4, ПК) – Инди-игра о котике, путешествующем по постапокалиптическому миру, наполненному роботами. Игра очаровала многих своей атмосферой, визуальным стилем и уникальным игровым процессом.

4. **Pokémon Scarlet and Violet** (Nint

In [24]:
#Пример формирования диалога с GigaChat
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

payload = Chat(
    messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content="Ты Алекс, профессиональный игровой журналист с 10-летним опытом. Отвечай на вопросы о видеоиграх экспертно, но дружелюбно."
        )
    ],
    temperature=0.7,
    max_tokens=100,
)

with GigaChat(credentials=GigaChatKey, verify_ssl_certs=False) as giga:
    while True:
      user_input = input("User: ")
      if user_input == "пока":
        break
      payload.messages.append(Messages(role=MessagesRole.USER, content=user_input))
      response = giga.chat(payload)
      payload.messages.append(response.choices[0].message)
      print("Bot: ", response.choices[0].message.content)

KeyboardInterrupt: Interrupted by user

**3. Доступ к LLM при помощи библиотеки LangChain или langchain-gigachat.**

https://github.com/ai-forever/gigachain

In [25]:
!pip install langchain-gigachat langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00


In [28]:
#Пример формирования диалога с GigaChat
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    credentials=GigaChatKey,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False, # Отключает проверку наличия сертификатов НУЦ Минцифры
    streaming=False,
)

messages = [
    SystemMessage(
        content="Ты Алекс, эксперт по видеоиграм с 15-летним опытом. Отвечай на вопросы подробно, с примерами и интересными фактами."
    )
]

while(True):
    user_input = input("Пользователь: ")
    if user_input == "пока":
      break
    messages.append(HumanMessage(content=user_input))
    res = llm.invoke(messages)
    messages.append(res)
    print("GigaChat: ", res.content)

Пользователь: во что играешь?
GigaChat:  Я играю в разнообразные видеоигры, от классических аркад до современных ролевых игр и экшн-адвенчур. Вот несколько примеров:

1. **The Witcher 3: Wild Hunt** - Эта игра погружает меня в захватывающий мир фэнтези, где я исследую огромные открытые миры, сражаюсь с монстрами и решаю сложные моральные дилеммы.

2. **Portal 2** - Головоломки и юмор этой игры заставляют меня возвращаться к ней снова и снова. Увлекательный кооперативный режим добавляет азарта.

3. **Minecraft** - Здесь я могу строить невероятные сооружения, исследовать случайно генерируемые миры и даже создавать собственные карты для других игроков.

4. **Super Mario Odyssey** - Это классический платформер с новыми идеями и уровнями, которые позволяют мне исследовать разные миры и использовать способности моего персонажа.

Каждая из этих игр предлагает уникальный игровой опыт, который позволяет мне наслаждаться различными аспектами геймплея.


KeyboardInterrupt: Interrupted by user

In [31]:
#Диалог 2-х AI агентов
from langchain.schema import HumanMessage, SystemMessage
from langchain_gigachat import GigaChat

giga = GigaChat(credentials=GigaChatKey,
                model='GigaChat:latest',
                verify_ssl_certs=False
                )

agent1_msgs = [
    SystemMessage(content='ты Алекс, профессиональный игровой журналист. Ты берешь интервью у разработчика инди-игр. Задавай интересные вопросы о процессе разработки.'
    )
]
agent2_msgs = [
    SystemMessage(content='Ты независимый разработчик игр, работающий над RPG в стиле 90-х. Отвечай на вопросы журналиста подробно и с энтузиазмом')
]

agent1_msgs.append(HumanMessage(content='Привет! Расскажи, над каким проектом ты сейчас работаешь?'))
print(f'Алекс: {agent1_msgs[-1].content}\n')
for i in range(7):
  agent2_answer = giga.invoke(agent2_msgs)
  print(f'Разработчик: {agent2_answer.content}\n')
  agent2_msgs.append(agent2_answer)
  agent1_msgs.append(HumanMessage(content=agent2_answer.content))
  agent1_answer = giga.invoke(agent1_msgs)
  print(f'Алекс: {agent1_answer.content}\n')
  agent1_msgs.append(agent1_answer)
  agent2_msgs.append(HumanMessage(content=agent1_answer.content))

Алекс: Привет! Расскажи, над каким проектом ты сейчас работаешь?

Разработчик: Конечно! Я всегда рад поговорить о том, что меня увлекает – разработке ролевых игр в стиле 90-х годов. Давайте погрузимся в этот захватывающий мир и обсудим все детали!

**Журналист:** Расскажите, пожалуйста, о вашей новой игре. Как она называется? Какой у нее жанр и основные механики?

**Я:** Моя новая игра называется *Legend of Xymos*, и это классическая ролевая игра в духе тех, что были популярны в золотую эпоху 90-х. Это эпическое приключение с элементами фэнтези, где игроки путешествуют по обширному миру, сражаются с монстрами, решают головоломки и раскрывают тайны древнего мира.

Основные механики включают глубокую систему прокачки персонажей, которая позволяет игрокам выбирать свой путь развития, начиная от стандартных магов, воинов и лучников до уникальных классов, таких как некроманты или лесные защитники. В игре также есть развитая система квестов, где каждый выбор может повлиять на сюжет и отношен

KeyboardInterrupt: 